In [2]:
import numpy as np
import pandas as pd  
import matplotlib.pyplot as plt 
import seaborn as sns
import warnings
from scipy.stats import pearsonr 
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from statsmodels.tsa.arima.model import ARIMA
import xgboost as xgb
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout, Bidirectional
from math import sqrt
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
import tensorflow as tf
from tensorflow.keras.models import Sequential
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.layers import Dense, LSTM, Dropout, Bidirectional
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.preprocessing import MinMaxScaler

2024-04-02 00:18:37.066275: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
def json_to_prices(json):
    df = pd.read_json(json)
    prices = df['prices']
    coin_prices = pd.DataFrame(prices.tolist(), columns=['time', 'price'])
    prices = coin_prices['price']
    return prices

In [4]:
def normalize_prices(prices):
    normalized_prices = (prices - np.mean(prices)) / np.std(prices)
    return normalized_prices

def split_data(normalized_prices, split=0.8):
    train_size = int(len(normalized_prices) * split)
    test_size = len(normalized_prices) - train_size
    
    training_data = normalized_prices[0:train_size]
    test_data = normalized_prices[train_size:len(normalized_prices)]
    return training_data, test_data

def create_dataset_helper(data, time_step=1):
    X, y = [], []
    # data = data.reset_index(drop=True)

    for i in range(len(data) - time_step):      
        X.append(data[i:(i + time_step)])
        y.append(data[i + time_step])
    return np.array(X), np.array(y)

def create_dataset(data, time_steps=1):
    X, y = create_dataset_helper(data, time_steps)
    X = X.reshape(X.shape[0], X.shape[1], 1)
    return X, y

def create_sequences(data, time_steps):
    X, y = [], []
    for i in range(time_steps, len(data)):
        X.append(data[i-time_steps:i])
        y.append(data[i, 0])  # Assuming the first column is DAI price
    return np.array(X), np.array(y)

In [5]:
def create_model(features, time_step):
    model = Sequential()
    model.add(LSTM(50, return_sequences=True, input_shape=(time_step, features)))
    model.add(LSTM(50, return_sequences=False))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

# def create_multivariate_model():
#     model = Sequential()
#     model.add(LSTM(units=50, return_sequences=True, input_shape=(time_step, 3)))
#     model.add(LSTM(units=50))
#     model.add(Dense(1))
#     model.compile(optimizer='adam', loss='mean_squared_error')
#     return model

def predictions(model, X_train, y_train, X_test, y_test, prices):
    train_predictions = model.predict(X_train)
    test_predictions = model.predict(X_test)
    
    train_predict = (train_predictions * np.std(prices)) + np.mean(prices)
    y_train = (y_train * np.std(prices)) + np.mean(prices)
    test_predict = (test_predictions * np.std(prices)) + np.mean(prices)
    y_test = (y_test * np.std(prices)) + np.mean(prices)
    
    train_score = sqrt(mean_squared_error(y_train, train_predict))
    print('Train Score: %.20f RMSE' % (train_score))
    test_score = sqrt(mean_squared_error(y_test, test_predict))
    print('Test Score: %.20f RMSE' % (test_score))
    return train_predict, test_predict
    

In [11]:
# Load the data

dai = json_to_prices('data/dai90days.json')
eth = json_to_prices('data/ethereum90days.json')
usdc = json_to_prices('data/usdc90days.json')

#normalize data

normalized_dai = normalize_prices(dai)
normalized_eth = normalize_prices(eth)
normalized_usdc = normalize_prices(usdc)


In [12]:
# combine the data

combined_data = pd.concat([normalized_dai, normalized_eth, normalized_usdc], axis=1)

In [13]:
# remove null values

combined_data = combined_data.dropna()

# remove column names
combined_data = combined_data.values
print(combined_data)

[[ 1.36337107 -1.50989476  0.31397376]
 [ 0.02828203 -1.49104902 -0.13996421]
 [ 0.67319998 -1.50721905 -0.70738667]
 ...
 [ 1.34129665  3.23881176  0.54094275]
 [ 2.75704907  3.19242724 -1.50177812]
 [ 0.69631532  3.21266332  0.08700478]]


In [14]:
#split the data
time_step = 10

combined_training_data, combined_test_data = split_data(combined_data)

X_train, y_train = create_sequences(combined_training_data, time_step)
X_test, y_test = create_sequences(combined_test_data, time_step)

In [15]:
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(1718, 10, 3) (1718,) (422, 10, 3) (422,)


In [16]:
model = create_model(3, time_step)
model.fit(X_train, y_train, epochs=50, batch_size=64)

Epoch 1/50
27/27 [==============================] - 3s 9ms/step - loss: 0.8891
Epoch 2/50
27/27 [==============================] - 0s 10ms/step - loss: 0.8518
Epoch 3/50
27/27 [==============================] - 0s 8ms/step - loss: 0.8398
Epoch 4/50
27/27 [==============================] - 0s 8ms/step - loss: 0.8343
Epoch 5/50
27/27 [==============================] - 0s 8ms/step - loss: 0.8285
Epoch 6/50
27/27 [==============================] - 0s 14ms/step - loss: 0.8205
Epoch 7/50
27/27 [==============================] - 0s 8ms/step - loss: 0.8206
Epoch 8/50
27/27 [==============================] - 0s 8ms/step - loss: 0.8092
Epoch 9/50
27/27 [==============================] - 0s 8ms/step - loss: 0.8037
Epoch 10/50
27/27 [==============================] - 0s 9ms/step - loss: 0.7975
Epoch 11/50
27/27 [==============================] - 0s 8ms/step - loss: 0.7974
Epoch 12/50
27/27 [==============================] - 0s 7ms/step - loss: 0.7919
Epoch 13/50
27/27 [============================

In [19]:
train_predict, test_predict = predictions(model, X_train, y_train, X_test, y_test, dai)

test_predict
test_predict.tolist()

14/14 [==============================] - 0s 4ms/step
Train Score: 0.00093989122757551279 RMSE
Test Score: 0.00141322645378592145 RMSE


[[0.9998248815536499],
 [0.9997718930244446],
 [0.9998355507850647],
 [1.0001161098480225],
 [1.000505805015564],
 [1.000762939453125],
 [1.0007435083389282],
 [1.000102162361145],
 [1.0000345706939697],
 [0.9994521141052246],
 [0.9992328882217407],
 [0.9991752505302429],
 [0.999911367893219],
 [0.9994857311248779],
 [0.9997633099555969],
 [1.0004228353500366],
 [0.9998823404312134],
 [0.999511182308197],
 [1.0001323223114014],
 [0.9994065761566162],
 [1.0000470876693726],
 [0.9996899366378784],
 [0.9995107650756836],
 [1.0003610849380493],
 [1.0002822875976562],
 [0.9999175071716309],
 [0.9998939633369446],
 [0.9995456337928772],
 [0.9997302889823914],
 [0.9991461038589478],
 [0.9986851215362549],
 [0.9993930459022522],
 [0.9992066025733948],
 [0.9992901086807251],
 [0.9991307854652405],
 [0.9993988871574402],
 [0.9999228119850159],
 [0.9994692206382751],
 [0.999198853969574],
 [0.9995038509368896],
 [0.999628484249115],
 [0.9997326731681824],
 [0.9996628761291504],
 [0.99996739625930

In [48]:
def output_to_list(json, array):
    df = pd.read_json(json)
    prices = df['prices']
    coin_prices = pd.DataFrame(prices.tolist(), columns=['time', 'price'])
    timeprev = coin_prices['time']
    
    latest_timestamp = timeprev.max()
    #print(latest_timestamp)
    
    num_predictions = len(array)
    predictions_timestamps = [latest_timestamp + i for i in range(1, num_predictions+1)]
    
    print(type(predictions_timestamps))
    print(type(array))
    
    predictions_df = pd.DataFrame({'timestamp': predictions_timestamps, 'predicted price': array.tolist()})
    print(predictions_df)
    return prices

In [49]:
output_to_list('data/dai90days.json', test_predict)

<class 'list'>
<class 'numpy.ndarray'>
         timestamp       predicted price
0    1709686849001  [0.9998248815536499]
1    1709686849002  [0.9997718930244446]
2    1709686849003  [0.9998355507850647]
3    1709686849004  [1.0001161098480225]
4    1709686849005   [1.000505805015564]
..             ...                   ...
417  1709686849418  [0.9991946816444397]
418  1709686849419  [0.9991279244422913]
419  1709686849420  [0.9990339875221252]
420  1709686849421  [0.9992547035217285]
421  1709686849422  [0.9994848370552063]

[422 rows x 2 columns]


0         [1701910823302, 1.00099352038083]
1       [1701914449766, 0.9993816102823021]
2        [1701918067065, 1.000160247309781]
3         [1701921642457, 0.99957610205443]
4       [1701925236930, 0.9996380997231781]
                       ...                 
2156      [1709672460735, 1.00072776876598]
2157      [1709676034383, 1.00096686898166]
2158     [1709679606219, 1.002676167420901]
2159     [1709683362679, 1.000188155447353]
2160     [1709686849000, 1.000742111037462]
Name: prices, Length: 2161, dtype: object